# Mix

In [33]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import matplotlib.pyplot as plt

from scipy import ndimage 

import os
import cv2
import pandas as pd
%matplotlib inline

In [34]:
#Load the coordinates data set
df = pd.read_csv('train.csv')

In [35]:
# Load images and resize 
root= './train'
names = []
train_crude = []
for file in os.listdir(root):
    img = cv2.imread(os.path.join(root, file))#,cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (img.shape[1]//3,img.shape[0]//3))
    train_crude.append(img)
    names.append(file.split('.')[0])
    
data = np.stack( train_crude, axis=0 )

### Extract feature CNN

In [ ]:
tf.__version__

In [36]:
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
base_model = ResNet50V2(weights='imagenet',input_shape=[163, 226,3],include_top=False)

In [37]:
#get features in batches and flatten
feat = []
for i in range(0,7500,500):
    temp = base_model.predict(preprocess_input(data[i:i+500]))
    feat.append(temp)
    

feat0 = np.concatenate(feat, axis=0) 
feat2 = [i.ravel() for i in feat0]
feat2 = np.stack(feat2, axis=0) #these are the features

In [ ]:
feat2.shape

### Nearest neighbor

In [38]:

from sklearn.neighbors import NearestNeighbors
train = feat2[:7000]
validation = feat2[7000:]

neighTest = NearestNeighbors(n_neighbors=5)
neighTest.fit(train)

predictions = neighTest.kneighbors(validation)

In [39]:
predictions[0][0] # Distances

array([437.69397, 452.00183, 455.92117, 459.7137 , 459.8171 ],
      dtype=float32)

In [40]:
predictions[1][0] # Indices

array([2964,  684,  322, 3702, 6431], dtype=int64)

In [ ]:
plt.subplots(figsize=(15,15))
for ix,im in enumerate(predictions):
    toShow = data[im]
    plt.subplot(1,6,ix+1)
    plt.imshow(toShow, cmap='gray')  
    plt.title(im)
    plt.axis('off')
plt.show()

### SIFT

In [ ]:
sift = cv2.SIFT_create()

#Generates de key points and descriptors of the neighbours of the images to be predicted

key_points=[]
descript=[]
for  i, i_n in enumerate(predictions[1]): #Iterate over indices of predictions
    # print(i_n)
    kp_neighbours=[]
    desc_neighbours=[]
    for neigh in i_n: # Iterates over the indices of the k nearest neighbours of the instance being predicted
        neighbour=data[neigh] # A neighbour of the instances being predicted
        kp, des = sift.detectAndCompute(neighbour,None) # key points and descriptors of the neighbour of instance being predicted
        kp_neighbours.append(kp)
        desc_neighbours.append(des)
        
    key_points.append(kp_neighbours)
    descript.append(desc_neighbours)



In [122]:
len(descript[0]) #[0][0]

5

In [103]:
#Generates de key points and descriptors of the images to be predicted

validation_img=data[7000:]

key_points_dev=[]
descript_dev=[]

for  img in validation_img: #Iterate over indices of predictions
    # print(i_n)
    kp, des = sift.detectAndCompute(img,None) # key points and descriptors of the instance being predicted
    key_points_dev.append(kp)
    descript_dev.append(des)
    
#descript_dev=np.array(descript_dev)

C:\Users\Alex\.conda\envs\CV\lib\site-packages\ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == '':


In [118]:
len(descript_dev[0][0])

128

In [145]:
def find_matches(matcher,des1, des2):
    # Matching descriptor using KNN algorithm
    print('\n\n')
    print(des1,'\n')
    print(des2)
    matches = matcher.knnMatch(des1,des2,k=2)

    # Store all good matches as per Lowe's Ratio test.
    ratio = 0.6
    good = []
    for m,n in matches:
        if m.distance < ratio*n.distance:
            good.append(m)
    return len(good)

def match(desc_list1, desc_list2):
    
    # FLANN parameters and initialize
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)   # or pass empty dictionary
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    
    best_neighbours_matched=[]
    for i, desc1 in enumerate(desc_list1):
        
        print('image {}'.format(i))
        matches_list=[]
        ind_neighbour=[]
        for j, desc2 in enumerate(desc_list2[i]):
            #print(j)
            if desc2 is not None:
                print('neighb {}'.format(j))
                ind_neighbour.append(predictions[1][i][j]) #Appends index in the data of the neighbour from the image being predicted
                n_matches = find_matches(flann, desc1, desc2)
                matches_list.append(n_matches) #Appends matches between the current neighbour and the instance being predicted
        
        #print('len ind neigh',ind_neighbour)
        #print('len matches list',matches_list)
        neighb_matches=np.stack([ind_neighbour,matches_list],axis=1) #Creates numpy array. Rows are descriptors of different neighbours. 
                                                                     # First column is index in the data, Second column is how many matches that neighbour has with the current instance
        neighb_matches=neighb_matches[neighb_matches[:, 1].argsort()[::-1][:len(matches_list)]]  #Sorts array in descending order, more matches on top and less matches at the bottom :5 is the number of neighbours
        #print(neighb_matches)
        best_neighbours_matched.append(neighb_matches)
    
    return best_neighbours_matched
                                 
k_best_matches=match(descript_dev,descript)

image 0
neighb 0



[[ 0.  0.  0. ...  2. 16.  2.]
 [ 0.  0.  1. ...  5. 12. 19.]
 [45. 12.  0. ...  0.  0.  0.]
 ...
 [ 2.  0.  1. ...  0.  0.  0.]
 [ 0.  0.  1. ...  0.  0.  1.]
 [ 2.  0.  0. ...  0.  4. 12.]] 

[[  5.   1.  15. ...   2.  27.  20.]
 [  1.  10.  14. ...   1.   0.   0.]
 [  4.   9.   7. ...  12.   0.   0.]
 ...
 [ 14.   2.   1. ...   3.   9.   3.]
 [  1.  49.  35. ...   0.   0.   0.]
 [ 86. 108.   1. ...   0.   0.   0.]]
neighb 1



[[ 0.  0.  0. ...  2. 16.  2.]
 [ 0.  0.  1. ...  5. 12. 19.]
 [45. 12.  0. ...  0.  0.  0.]
 ...
 [ 2.  0.  1. ...  0.  0.  0.]
 [ 0.  0.  1. ...  0.  0.  1.]
 [ 2.  0.  0. ...  0.  4. 12.]] 

[[ 0.  0. 63. ...  2.  0.  2.]
 [ 0.  0. 62. ... 12.  2.  9.]
 [39.  1.  0. ...  1.  3.  1.]
 ...
 [ 1. 21. 66. ...  7.  4. 31.]
 [ 2.  0.  0. ...  5.  2.  4.]
 [ 0.  5. 76. ...  0.  0.  0.]]
image 1
neighb 2



[[11.  0.  0. ...  3.  2.  6.]
 [ 3. 74. 35. ... 14. 12.  1.]
 [36. 95. 49. ... 25. 25.  0.]
 ...
 [59.  0.  0. ...  0.  0.  0.]
 [ 0.  1.  

error: OpenCV(4.5.2) ..\modules\flann\src\miniflann.cpp:521: error: (-215:Assertion failed) (size_t)knn <= index_->size() in function 'cv::flann::runKnnSearch_'
